In [1]:
import pandas as pd

columns = ['id','text','label']
train_data = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', sep='\t', names=columns, skiprows=1).dropna()
test_data = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', sep='\t', names=columns, skiprows=1).dropna()

In [3]:
train_data.to_csv('C:\\Users\\admin\\jupyter\\pytorch\\Sentiment_analysis\\train_data.csv', index=False)
test_data.to_csv('C:\\Users\\admin\\jupyter\\pytorch\\Sentiment_analysis\\test_data.csv', index = False)

In [4]:
import torch
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [7]:
from transformers import BertTokenizer

# bert_base_multilingual-cased : 트랜스포어의 사전 훈련 벡터
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [8]:
len(tokenizer.vocab)

119547

In [9]:
# “##”는 해당 심볼을 지닌 토큰은 해당 토큰 이전에 등장한 토큰과 공백 없이 합쳐져야 한다는 의미
tokens = tokenizer.tokenize('우리 사이엔 낮은 담이 있어 서로의 진심을 안을 수가 없어요')
print(tokens)

['우', '##리', '사', '##이', '##엔', '낮', '##은', '담', '##이', '있어', '서로', '##의', '진', '##심을', '안', '##을', '수', '##가', '없', '##어', '##요']


In [10]:
# tokenizer.convert_tokens_to_ids : 토큰을 인덱스로 변경
indexes = tokenizer.convert_tokens_to_ids(tokens)
print(indexes)

[9604, 12692, 9405, 10739, 86933, 8992, 10892, 9064, 10739, 45893, 67324, 10459, 9708, 86904, 9521, 10622, 9460, 11287, 9555, 12965, 48549]


In [11]:
init_token = tokenizer.cls_token # cls_token : 입력 클래스 토큰
eos_token = tokenizer.sep_token # sep_token : 동일한 입력에서 두 개의 다른 문장을 분리
pad_token = tokenizer.pad_token # pad_token : 패딩 토큰
unk_token = tokenizer.unk_token # unk_token : 미지정 토큰

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


In [12]:
# 각 토큰들의 인덱스 확인
nit_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [13]:
# max_model_input_sizes : 트랜스포머 모델의 문장 최대 길이
max_input_length = tokenizer.max_model_input_sizes['bert-base-multilingual-cased']
print(max_input_length)

512


In [14]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    return tokens

In [16]:
from torchtext import data

# 트랜스포머 모델은 배치 차원을 맨 앞에 위치해야 함.
TEXT = data.Field(batch_first = True, use_vocab = False, tokenize = tokenize_and_cut,
                 preprocessing = tokenizer.convert_tokens_to_ids, init_token = init_token_idx,
                 eos_token = eos_token_idx,
                 pad_token = pad_token_idx,
                 unk_token = unk_token_idx)

LABEL = data.LabelField(dtype=torch.float)

In [17]:
fields = {'text':('text', TEXT), 'label':('label', LABEL)}

In [18]:
train_data, test_data = data.TabularDataset.splits(
                            path = 'C:\\Users\\admin\\jupyter\\pytorch\\Sentiment_analysis',
                            train = 'train_data.csv',
                            test = 'test_data.csv',
                            format = 'csv',
                            fields = fields,  
)
train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [19]:
print(vars(train_data.examples[10]))

{'text': [9574, 119439, 10459, 18622, 42815, 12638, 30842, 14423, 48418, 11664, 80331, 10622, 9056, 35866, 11018, 119259, 12178, 117, 100, 119, 9428, 12030, 85903, 16439, 9729, 26737, 17342], 'label': '0'}


In [20]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[10])['text'])
print(tokens)

['영', '##혼', '##의', '##치', '##유', '##와', '##현', '##대', '##인의', '##고', '##독', '##을', '다', '##루', '##는', '##척', '##하는', ',', '[UNK]', '.', '선', '##인', '##장에', '##나', '찔', '##려', '##라']


In [21]:
string = tokenizer.convert_tokens_to_string(tokens)
print(string)

영혼의치유와현대인의고독을 다루는척하는 , [UNK] . 선인장에나 찔려라


In [22]:
LABEL.build_vocab(train_data)

In [23]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True,
    device = device)

# 모델 생성

In [25]:
from transformers import BertModel

bert = BertModel.from_pretrained('bert-base-multilingual-cased')

In [26]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self, bert, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()
        self.bert = bert
        embedding_dim = bert.config.to_dict()['hidden_size']
        self.rnn = nn.GRU(embedding_dim, hidden_dim, num_layers = n_layers,
                         bidirectional = bidirectional, batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        with torch.no_grad():
            embedded = self.bert(text)[0]
        
        _, hidden = self.rnn(embedded)
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        
        output = self.out(hidden)
        
        return output

In [27]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert, HIDDEN_DIM, OUTPUT_DIM,
                        N_LAYERS, BIDIRECTIONAL, DROPOUT)

In [28]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'모델의 파라미터 수는 {count_parameters(model):,}, \
이 중 버트 모델의 파라미터 수는 {count_parameters(bert):,}개입니다.')

모델의 파라미터 수는 180,612,609, 이 중 버트 모델의 파라미터 수는 177,853,440개입니다.


In [29]:
for name, param in model.named_parameters():

    if name.startswith('bert'):
        param.requires_grad = False

In [30]:
print(f'모델의 파라미터 수는 {count_parameters(model):,}개입니다.')

모델의 파라미터 수는 2,759,169개입니다.


In [31]:
for name, param in model.named_parameters():

    if param.requires_grad == True:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


# 모델 훈련, 검증 함수 생성

In [32]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

In [33]:
model = model.to(device)
criterion = criterion.to(device)

In [34]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [35]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1) 
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [36]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [37]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [38]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 2m 23s
	Train Loss: 0.517 | Train Acc: 73.78%
	 Val. Loss: 0.468 |  Val. Acc: 77.73%
Epoch: 02 | Epoch Time: 2m 27s
	Train Loss: 0.424 | Train Acc: 80.00%
	 Val. Loss: 0.388 |  Val. Acc: 82.08%
Epoch: 03 | Epoch Time: 2m 37s
	Train Loss: 0.384 | Train Acc: 82.40%
	 Val. Loss: 0.377 |  Val. Acc: 83.02%
Epoch: 04 | Epoch Time: 2m 33s
	Train Loss: 0.359 | Train Acc: 83.64%
	 Val. Loss: 0.381 |  Val. Acc: 82.95%
Epoch: 05 | Epoch Time: 2m 35s
	Train Loss: 0.336 | Train Acc: 84.84%
	 Val. Loss: 0.362 |  Val. Acc: 83.76%


In [39]:
model.load_state_dict(torch.load('tut6-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.362 | Test Acc: 83.84%


In [40]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [41]:
predict_sentiment(model, tokenizer, "내 인생 영화 등극. 주인공한테 너무 몰입해서 시간 가는 줄도 몰랐다...")

0.883232057094574